# Code pour reformater les vidéos au même nombre de frames pour faciliter l'extraction et le traitement

# 1. Imports

In [26]:
import cv2
import os
import numpy as np
from os.path import isfile, join
from moviepy.editor import VideoFileClip

# 2. Paramétrer les dossiers

Paramètres de bases

In [27]:
path = '../mots/' #path vers le dossier des mots
data_path = 'new_videos/'
mots=np.array(['adresse','affaire']) #choix des mots
nb_videos=20 #nb de vidéos par mot

Calculer le nombre d'images moyen par vidéo pour avoir la référence

In [28]:
count=0
fichier =list([])
for mot in mots:
    fichier1=[]
    i=0
    for f in os.listdir(path):
        if i<nb_videos:
            file_name, file_ext = os.path.splitext(f)
            testmot = file_name.split('-')
            if (testmot[0]==mot):
                fichier1 += [path + file_name + file_ext]
                i+=1
    fichier.append(fichier1)
rang_mot=0
for mot in mots:
    for f in fichier[rang_mot]:
        cap = cv2.VideoCapture(f)
        length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        count=count+length
    rang_mot+=1
moy=count/(len(fichier)*nb_videos)
nb_frame=int(moy)#nombre d'images par vidéos
print(nb_frame)

20


Création des dossiers pour le stockage des images & vidéos

In [29]:
for mot in mots: 
    for vid in range(nb_videos):
        try: 
            os.makedirs(os.path.join(data_path, mot, str(vid)))
        except:
            pass

# 3. Code

Fonction intermédiaire pour plus tard calculer le framerate nécessaire à l'obtention du bon nombre de frames

In [35]:
def duree_et_longueurCV(video):

    duree = video.get(cv2.CAP_PROP_POS_MSEC)
    longueur = video.get(cv2.CAP_PROP_FRAME_COUNT)

    return duree, longueur
def dureeMPY(file):
    clip = VideoFileClip(file)
    return clip.duration

Fonction pour stocker une image

In [31]:
def getFrame(vidcap,sec,rang_video,count):
            vidcap.set(cv2.CAP_PROP_POS_MSEC,sec*1000)
            hasFrames,image = vidcap.read()
            if hasFrames:
                if count<10:
                    numero = '0' + str(count)
                else :
                    numero = str(count)
                cv2.imwrite(data_path+mot+'/'+str(rang_video)+'/image'+numero+".jpg", image)     # save frame as JPG file
            return hasFrames

Convertir un .avi en .mp4 pour respecter le format donné

In [32]:
def convert_avi_to_mp4(avi_file_path, output_name):
    os.popen("ffmpeg -i '{input}' -ac 2 -b:v 2000k -c:a aac -c:v libx264 -b:a 160k -vprofile high -bf 0 -strict experimental -f mp4 '{output}.mp4'".format(input = avi_file_path, output = output_name))
    return True

Code pour stocker automatiquement toutes les images des mots nous intéressants

In [36]:
rang_mot=0
for mot in mots:
    #print(mot)
    rang_video=0
    for f in fichier[rang_mot]:
        #print(f)
        sec = 0
        count=1
        #print(data_path+mot+'/'+str(rang_video)+'/image'+str(count)+".jpg")
        vidcap = cv2.VideoCapture(f)
        duree = dureeMPY(f)
        print(duree)
        framerate = (duree/nb_frame) # RESTE A DEFINIR EN FONCTION DE LA VIDEO, A VOIR COMMENT FAIRE
        success = getFrame(vidcap,sec,rang_video,count)
        while success:
            count = count + 1
            sec = sec + framerate
            sec = round(sec, 2)
            success = getFrame(vidcap,sec,rang_video,count)
        print(len(os.listdir(data_path+mot+'/'+str(rang_video)+'/')))
        rang_video+=1
    rang_mot+=1

0.92
18
0.96
19
1.04
21
0.92
18
0.96
19
0.92
18
0.88
22
0.84


KeyboardInterrupt: 

Code pour fusionner nos images en nouvelles vidéos avec le bon nombre de frame

In [102]:
for mot in mots:
    rang_video=0
    for j in range (nb_videos):
        pathIn= data_path + mot + '/' + str(rang_video) + '/'
        pathtodelete=data_path + mot + '/' + str(rang_video)
        pathOut = data_path + mot + '/' + mot + '-' + str(rang_video) + '.mp4'
        fps = 10
        frame_array = []
        files = [f for f in os.listdir(pathIn) if isfile(join(pathIn, f))]
        files.sort()
        for i in range(len(files)):
            img = cv2.imread(pathIn + files[i])
            height, width, layers = img.shape
            size = (width,height)
            frame_array.append(img) #insérer une nouvelle image dans la vidéo
        out = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'mp4v'), fps, size)
        for i in range(len(frame_array)):
            #écriture de la vidéo
            out.write(frame_array[i])
        out.release()
        #supprimer tous les fichiers images maintenant inutiles
        for f in files:
            os.remove(pathIn+f)
        os.rmdir(pathtodelete)
        rang_video+=1